In [2]:
from IPython.display import IFrame
IFrame(src= "https://dash-simple-apps.plotly.host/dash-networkplot/", width="100%", height="650px", frameBorder="0")

In [1]:
from numpy import  array,count_nonzero
from scipy.sparse import csr_matrix,csc_matrix,diags
import numpy as np
import time

In [2]:
# create dense matrix
A = array([[1, 0, 0], [ 2,0, 1], [2, 0, 0]])
print('array:',A)
S=csr_matrix(A)
print('sparse:',S)
B=S.todense()
print('dense:',B)
print(type(S))
# print(isinstance(S,scipy.sparse.csr.csr_matrix))

array: [[1 0 0]
 [2 0 1]
 [2 0 0]]
sparse:   (0, 0)	1
  (1, 0)	2
  (1, 2)	1
  (2, 0)	2
dense: [[1 0 0]
 [2 0 1]
 [2 0 0]]
<class 'scipy.sparse.csr.csr_matrix'>


In [47]:
#指定索引后创建一个
a=csr_matrix(([1,2,3,4],([0,1,2,3],[3,2,1,0])),shape=(4,4))
print(a.toarray())

[[0 0 0 1]
 [0 0 2 0]
 [0 3 0 0]
 [4 0 0 0]]


In [48]:
b=a.tocoo()
print(b.row)
print(b.col)
print(b.data)

[0 1 2 3]
[3 2 1 0]
[1 2 3 4]


In [69]:
import torch
ii=np.stack((b.row,b.col),axis=0)
ii=torch.from_numpy(ii).long()
print(ii)
d=torch.from_numpy(b.data)
print(d)
torch.sparse.FloatTensor(indices=ii,values=d,device='cpu')
# torch.sparse.FloatTensor(indices=torch.Tensor([[0,1]]),values=torch.Tensor([1]),device='cpu')

tensor([[0, 1, 2, 3],
        [3, 2, 1, 0]])
tensor([1, 2, 3, 4])


tensor(indices=tensor([[0, 1, 2, 3],
                       [3, 2, 1, 0]]),
       values=tensor([1, 2, 3, 4]),
       size=(4, 4), nnz=4, layout=torch.sparse_coo)

In [43]:
torch.Size(b.shape)

torch.Size([4, 4])

In [36]:
torch.from_numpy(ii)

tensor([[0, 1, 1, 2],
        [3, 1, 2, 1]], dtype=torch.int32)

In [36]:
sparse=1-count_nonzero(A)/A.size
print('sparse of A is :',sparse)

sparse of A is : 0.5555555555555556


In [38]:
#测试 矩阵乘法
r=S.dot(S.T)
print(r.todense())
r=A.dot(A.T)
print(r)
r=S.dot(csc_matrix([1,2,3]).T)
print(r)

[[1 2 2]
 [2 5 4]
 [2 4 4]]
[[1 2 2]
 [2 5 4]
 [2 4 4]]
  (0, 0)	1
  (1, 0)	5
  (2, 0)	2


In [41]:
D=diags([1,2,3])
print(D.shape)
print(S.shape)
print(S+D)

(3, 3)
(3, 3)
  (0, 0)	2.0
  (1, 0)	2.0
  (1, 1)	2.0
  (1, 2)	1.0
  (2, 0)	2.0
  (2, 2)	3.0


In [58]:
a=diags([1,2,3])
print(a)
b=S.sum(axis=1)
print(np.squeeze(np.asarray(b)))
print(diags(np.asarray(b).ravel()))

  (0, 0)	1.0
  (1, 1)	2.0
  (2, 2)	3.0
[1 3 2]
  (0, 0)	1.0
  (1, 1)	3.0
  (2, 2)	2.0


In [31]:
#测试速度
def test(N,nozero=1000):
    
    A=np.zeros(N*N)
    idx=np.random.choice(N*N,nozero,False)
    A[idx]=np.random.rand(len(idx))
    A=A.reshape(N,N)
    sparse=1-count_nonzero(A)/A.size
    print('sparse of A is :',sparse)
    
    start=time.time()
    A.dot(A)
    end=time.time()
    t1=end-start
    
    start=time.time()
    A=csr_matrix(A)
    A.dot(A)
    end=time.time()
    t2=end-start
    
    
    return t1,t2

In [36]:
N=2000
n=1
dense_history,sparse_history=[],[]
while n < N*N:
    t1,t2=test(N,n)
    n*=2
    dense_history.append(t1)
    sparse_history.append(t2)

sparse of A is : 0.99999975
sparse of A is : 0.9999995
sparse of A is : 0.999999
sparse of A is : 0.999998
sparse of A is : 0.999996
sparse of A is : 0.999992
sparse of A is : 0.999984
sparse of A is : 0.999968
sparse of A is : 0.999936
sparse of A is : 0.999872
sparse of A is : 0.999744
sparse of A is : 0.999488
sparse of A is : 0.998976
sparse of A is : 0.997952
sparse of A is : 0.995904
sparse of A is : 0.991808
sparse of A is : 0.983616
sparse of A is : 0.967232
sparse of A is : 0.934464
sparse of A is : 0.868928
sparse of A is : 0.7378560000000001
sparse of A is : 0.475712


In [41]:
#测色eigen value

In [49]:
from scipy.sparse.linalg import eigs
from numpy.linalg import eig

In [46]:
def randomSparse(N,nonzeros=0.1):
    A=np.zeros(N*N)
    nozero=int(N*N*nonzeros)
    idx=np.random.choice(N*N,nozero,False)
    A[idx]=np.random.rand(len(idx))
    A=A.reshape(N,N)
    sparse=1-count_nonzero(A)/A.size
    print('sparse of A is :',sparse)
    return A

In [63]:
K=6
A=randomSparse(1000)
S,V=eig(A)
print(S[:K])
print(V[0])

sparse of A is : 0.9
[49.89919161+0.j          5.18969976+1.95207416j  5.18969976-1.95207416j
  4.8133918 +2.61101211j  4.8133918 -2.61101211j  4.56141534+2.99615183j]
[-3.31156732e-02+0.00000000e+00j -1.75866098e-03+2.47911607e-03j
 -1.75866098e-03-2.47911607e-03j  6.22917851e-03-3.44447366e-02j
  6.22917851e-03+3.44447366e-02j -6.76511209e-03+7.30922429e-03j
 -6.76511209e-03-7.30922429e-03j  4.89938542e-03-2.19756513e-04j
  4.89938542e-03+2.19756513e-04j -6.01400516e-03-3.37420962e-02j
 -6.01400516e-03+3.37420962e-02j  1.96615542e-02+3.89652843e-02j
  1.96615542e-02-3.89652843e-02j -1.36258830e-02+8.11354769e-03j
 -1.36258830e-02-8.11354769e-03j  9.75904480e-03-6.08104974e-03j
  9.75904480e-03+6.08104974e-03j  1.50877173e-02-6.67065947e-03j
  1.50877173e-02+6.67065947e-03j  8.24666524e-03-2.49396718e-02j
  8.24666524e-03+2.49396718e-02j  1.15561954e-02-1.30580850e-02j
  1.15561954e-02+1.30580850e-02j  1.23698535e-02-2.29925351e-02j
  1.23698535e-02+2.29925351e-02j  1.41528379e-02+1.1

In [61]:
A=csr_matrix(A)
S,V=eigs(A,k=6,which='LR')

In [64]:
print(V[0])

[-3.31156732e-02+0.00000000e+00j -1.75866098e-03+2.47911607e-03j
 -1.75866098e-03-2.47911607e-03j  6.22917851e-03-3.44447366e-02j
  6.22917851e-03+3.44447366e-02j -6.76511209e-03+7.30922429e-03j
 -6.76511209e-03-7.30922429e-03j  4.89938542e-03-2.19756513e-04j
  4.89938542e-03+2.19756513e-04j -6.01400516e-03-3.37420962e-02j
 -6.01400516e-03+3.37420962e-02j  1.96615542e-02+3.89652843e-02j
  1.96615542e-02-3.89652843e-02j -1.36258830e-02+8.11354769e-03j
 -1.36258830e-02-8.11354769e-03j  9.75904480e-03-6.08104974e-03j
  9.75904480e-03+6.08104974e-03j  1.50877173e-02-6.67065947e-03j
  1.50877173e-02+6.67065947e-03j  8.24666524e-03-2.49396718e-02j
  8.24666524e-03+2.49396718e-02j  1.15561954e-02-1.30580850e-02j
  1.15561954e-02+1.30580850e-02j  1.23698535e-02-2.29925351e-02j
  1.23698535e-02+2.29925351e-02j  1.41528379e-02+1.19159627e-02j
  1.41528379e-02-1.19159627e-02j  2.45199786e-02+1.27709576e-02j
  2.45199786e-02-1.27709576e-02j -2.37410992e-02+5.43763725e-03j
 -2.37410992e-02-5.437637

In [59]:
##修改 PG

In [79]:
def pageRank(M,alpha=0.85,eps=0.1,maxiters=300):
    '''
        计算每个节点的 "价值",要求
        
        M:概率转换矩阵M[i,j]表示节点j到节点i的概率,sum(M[:,j])=1 
        
        V_t=alpha+(1-alpha)*M*V_t-1
        
        返回:降序返回
            V,表示每个节点的 "价值",
            rank:排名
    '''
    
    assert np.all(np.abs(np.sum(M,axis=0)-1)<1e-2),"矩阵M 的一列 之和 必须 为 1"
    
    t=0
    
    V=np.ones(M.shape[0])/M.shape[0]
    while t<maxiters:
        V_old=V
        if isinstance(M,np.ndarray):
            V=alpha+(1-alpha)*M.dot(V_old)
        else:
            p=M.dot(csr_matrix(V_old).T).toarray().ravel()
            V=alpha+(1-alpha)*p
        if np.max(np.abs(V-V_old))<eps:break
        t+=1
    sort_index=np.argsort(-V)
    V=V[sort_index]
    return V,sort_index

In [86]:
a=np.random.rand(4,3)
a=csr_matrix(a)
b=a.sum(axis=1)
# b=np.array([1,2,3])
# A=a.dot(csr_matrix(b).T)

In [87]:
a

<4x3 sparse matrix of type '<class 'numpy.float64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [89]:
b.shape

(4, 1)

In [94]:
c=a.multiply(1/b)

In [102]:
c.T.toarray().sum(axis=0)

array([1., 1., 1., 1.])

In [100]:
print(C.T)

NameError: name 'C' is not defined